In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

# Colaborative Filtering (CF)

## Model-Based
No tenemos features (Feature learning) tenemos que aprender tanto los  $\Theta$ como los $X$


$\huge  J = \frac{1}{2} \sum_{j=1}^{n_u} \sum_{i:r(i,j)=1} (\theta^{(j)^T}x^{(i)}-y^{(i,j)})^2 + \\
\huge \frac{\lambda}{2}\sum_{j=1}^{n_u} \sum_{k=1}^{n_f}\theta_k^{(j)^2} +
\frac{\lambda}{2}\sum_{i=1}^{n_m} \sum_{k=1}^{n_f}X_k^{(i)^2}$


$\huge J = \sum\limits_{(u,i)\in K}{(r_{ui} - \hat{r_{ui}})^2} + \lambda (||P_u||^2 + ||Q_i||^2) $  


**Algoritmos**:
- ALS (Alternated-Least-Squares)
- SGD (Stochastic Gradient Descent)
- SGD es más eficiente computacionalmente y suele tener mejor resultados salvo en casos de 'sparse data'
- http://cs229.stanford.edu/proj2014/Christopher%20Aberger,%20Recommender.pdf
- SVD (Singular Value Decomposition)

**Low Rank Matrix Factorization**:
Otra forma de pensar Model-Based CF y permite medir simitud entre peliculas o personas

## Baselines
- Media, Bu, Bi

#### Opcion 1
![baselines.png](baselines.png)

#### Opcion 2
![baselines_calc.png](baselines_calc.png)

#### Opcion 3

Dejarlos como features a calcular por el modelo

$\huge J = \sum\limits_{(u,i)\in K}{(r_{ui} - \mu - b_u - b_i - \hat{r_{ui}})^2} + \lambda (||P_u||^2 + ||Q_i||^2 + b_u^2 + b_i^2) $  

http://alex.smola.org/teaching/berkeley2012/slides/8_Recommender.pdf

# Ejemplo de bias

In [3]:
import numpy as np

In [4]:
R = np.array([[2, 0, 1, 3, 0], 
              [4, 0, 0, 5, 4],
              [0, 1, 3, 1, 4],
              [1, 5, 0, 2, 4],
              [0, 4, 2, 0, 4],
             ])
print(R)

[[2 0 1 3 0]
 [4 0 0 5 4]
 [0 1 3 1 4]
 [1 5 0 2 4]
 [0 4 2 0 4]]


In [5]:
mu = R[R.nonzero()].mean()
mu

2.9411764705882355

In [6]:
# Calculo de promedio de usuarios y promedio de pelis
print([R[:,i][R[:, i].nonzero()].mean() for i in range(5)])
print([R[i, :][R[i, :].nonzero()].mean() for i in range(5)])

[2.3333333333333335, 3.3333333333333335, 2.0, 2.75, 4.0]
[2.0, 4.333333333333333, 2.25, 3.0, 3.3333333333333335]


In [7]:
# Ratings restando la media
R_mu = (R - mu)*(R>0) 
R_mu

array([[-0.94117647, -0.        , -1.94117647,  0.05882353, -0.        ],
       [ 1.05882353, -0.        , -0.        ,  2.05882353,  1.05882353],
       [-0.        , -1.94117647,  0.05882353, -1.94117647,  1.05882353],
       [-1.94117647,  2.05882353, -0.        , -0.94117647,  1.05882353],
       [-0.        ,  1.05882353, -0.94117647, -0.        ,  1.05882353]])

In [8]:
# Redondeo para PPT
np.array(R_mu*100, dtype=int)/100

array([[-0.94,  0.  , -1.94,  0.05,  0.  ],
       [ 1.05,  0.  ,  0.  ,  2.05,  1.05],
       [ 0.  , -1.94,  0.05, -1.94,  1.05],
       [-1.94,  2.05,  0.  , -0.94,  1.05],
       [ 0.  ,  1.05, -0.94,  0.  ,  1.05]])

In [25]:
# Cálculo con librería
from cf_helper_2 import getBaselineEstimates
bui, bii = getBaselineEstimates(R, mu, items_first=True)
print(bui)
print(bii)

[[-0.36111111]
 [ 1.30555556]
 [-0.77083333]
 [-0.10416667]
 [ 0.22222222]]
[[-0.60784314]
 [ 0.39215686]
 [-0.94117647]
 [-0.19117647]
 [ 1.05882353]]


In [26]:
# Cálculo a mano

In [27]:
# Calculo bi que en el ppt es bu por que las matrices estan al reves
bi = [R_mu[:,i][R_mu[:, i].nonzero()].mean() for i in range(5)]
print(bi)

[-0.6078431372549021, 0.3921568627450978, -0.9411764705882355, -0.1911764705882355, 1.0588235294117645]


In [28]:
R_mu_bi = R_mu - bi

In [32]:
np.array(R_mu_bi*100, dtype=int)/100

array([[-0.33, -0.39, -1.  ,  0.25, -1.05],
       [ 1.66, -0.39,  0.94,  2.25,  0.  ],
       [ 0.6 , -2.33,  1.  , -1.75,  0.  ],
       [-1.33,  1.66,  0.94, -0.75,  0.  ],
       [ 0.6 ,  0.66,  0.  ,  0.19,  0.  ]])

In [46]:
bu_ = np.array([R_mu_bi[i, :][R[i, :].nonzero()].mean() for i in range(5)]).reshape(-1,1)
bu_

array([[-0.36111111],
       [ 1.30555556],
       [-0.77083333],
       [-0.10416667],
       [ 0.22222222]])

In [47]:
i = 4
R_mu_bi[i, :][R[i, :].nonzero()]

array([0.66666667, 0.        , 0.        ])

In [49]:
np.array((R_mu_bi - bu_)*100, dtype=int)/100

array([[ 0.02, -0.03, -0.63,  0.61, -0.69],
       [ 0.36, -1.69, -0.36,  0.94, -1.3 ],
       [ 1.37, -1.56,  1.77, -0.97,  0.77],
       [-1.22,  1.77,  1.04, -0.64,  0.1 ],
       [ 0.38,  0.44, -0.22, -0.03, -0.22]])

# PMF (Probabilistic Matrix Factorization)
Es equivalente a Matrix Factorization desde otro punto de vista


$\huge R = P_{mk}Q^T_{kn}$

Modelo generativo

- Rij -> Distribución gaussiana con media PiQj
- Pi -> Distribución gaussian con media cero. Probabilidad de que un usuario seleccione un topico
- Qj -> Distribución gaussian con media cero. Probabilidad de que dentro de un tópico se seleccione un item (No depende del usuario)
- El producto de P y Q equivale a la probabilidad de que un usuario elija un item ya que se suma en todos los tópicos K
- Tópico = Latent Feature

https://www.quora.com/What-is-the-difference-between-Probabilistic-Matrix-Factorization-PMF-and-SVD

# ¿Que diferencias tiene con NMF (Non-negative matrix factorization)?